In [ ]:
pip install langchain langchain_community langchain_groq sentence-transformers

In [ ]:
pip install pypdf chromadb

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/attention-is-all-you-need.pdf")
attention = loader.load()
attention[0].page_content

'Attention Is All Y ou Need\nAshish V aswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer ∗\nGoogle Brain\nnoam@google.com\nNiki Parmar ∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit ∗\nGoogle Research\nusz@google.com\nLlion Jones ∗\nGoogle Research\nllion@google.com\nAidan N. Gomez ∗ †\nUniversity of T oronto\naidan@cs.toronto.edu\nŁukasz Kaiser ∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin ∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. W e propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely . Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requirin

In [ ]:
loader = PyPDFLoader("/content/rag.pdf")
rag = loader.load()
rag[0].page_content

'1\nRetrieval-Augmented Generation for Large\nLanguage Models: A Survey\nYunfan Gaoa, Yun Xiongb, Xinyu Gao b, Kangxiang Jia b, Jinliu Pan b, Yuxi Bic, Yi Dai a, Jiawei Sun a, Meng\nWangc, and Haofen Wang a,c\naShanghai Research Institute for Intelligent Autonomous Systems, Tongji University\nbShanghai Key Laboratory of Data Science, School of Computer Science, Fudan University\ncCollege of Design and Innovation, Tongji University\nAbstract—Large Language Models (LLMs) showcase impres-\nsive capabilities but encounter challenges like hallucination,\noutdated knowledge, and non-transparent, untraceable reasoning\nprocesses. Retrieval-Augmented Generation (RAG) has emerged\nas a promising solution by incorporating knowledge from external\ndatabases. This enhances the accuracy and credibility of the\ngeneration, particularly for knowledge-intensive tasks, and allows\nfor continuous knowledge updates and integration of domain-\nspecific information. RAG synergistically merges LLMs’ intrin-

In [ ]:
len(attention), len(rag)

(11, 21)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter( chunk_size=500, chunk_overlap=100)


attention_split = splitter.split_documents(attention)
rag_split = splitter.split_documents(rag)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-8-1961396532.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import Chroma
import chromadb
import os

In [ ]:
os.getcwd()

'/content'

In [ ]:
current_dir = os.path.dirname(os.path.abspath("."))
current_dir

'/'

In [ ]:
DB_dir = os.path.join(current_dir, "/content/db")
DB_dir

'/content/db'

In [ ]:
client_settings = chromadb.config.Settings(
    is_persistent=True,
    persist_directory=DB_dir,
    anonymized_telemetry=False
)

In [ ]:
attention_db = Chroma.from_documents(
    attention_split,
    hf_embeddings,
    collection_name="attention",
    client_settings=client_settings,
    collection_metadata={"hnsw": "attention-is-all-you-need.pdf"},
    persist_directory="/store/attention"
)

In [ ]:
rag_db = Chroma.from_documents(
    rag_split,
    hf_embeddings,
    collection_name="rag",
    client_settings=client_settings,
    collection_metadata={"hnsw": "rag.pdf"},
    persist_directory="/store/rag"
)

In [ ]:
attention_retriever = attention_db.as_retriever(search_type = "mmr",search_kwargs={"k": 5, "include_metadata": True})
rag_retriever = rag_db.as_retriever(search_type = "mmr",search_kwargs={"k": 5, "include_metadata": True})

In [ ]:
from langchain.retrievers.merger_retriever import MergerRetriever

retriever = MergerRetriever(
    retrievers=[attention_retriever, rag_retriever]
)

In [ ]:
for chunks in retriever.get_relevant_documents("What are the main contributions of the paper 'Attention is All You Need'?"):
    print(chunks.page_content)
    print("\n\n")

3.2 Attention
An attention function can be described as mapping a query and a set of key-value pairs to an output,
where the query , keys, values, and output are all vectors. The output is computed as a weighted sum
of the values, where the weight assigned to each value is computed by a compatibility function of the
query with the corresponding key .
3.2.1 Scaled Dot-Product Attention
W e call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of



streaming language models with attention sinks,” arXiv preprint
arXiv:2309.17453, 2023.
[173] T. Zhang, S. G. Patil, N. Jain, S. Shen, M. Zaharia, I. Stoica, and J. E.
Gonzalez, “Raft: Adapting language model to domain specific rag,”
arXiv preprint arXiv:2403.10131 , 2024.
[174] J. Kaplan, S. McCandlish, T. Henighan, T. B. Brown, B. Chess,
R. Child, S. Gray, A. Radford, J. Wu, and D. Amodei, “Scaling laws
for neural language models,” arXiv preprint arXiv:2001.08361 , 2020.



[19] Zhouhan Lin, Minwei Feng

In [ ]:
for chunks in retriever.get_relevant_documents("What is RAG?"):
    print(chunks.page_content)
    print("\n\n")

summarization. arXiv preprint arXiv:1705.04304, 2017.
[24] Oﬁr Press and Lior W olf. Using the output embedding to improve language models. arXiv
preprint arXiv:1608.05859, 2016.
[25] Rico Sennrich, Barry Haddow , and Alexandra Birch. Neural machine translation of rare words
with subword units. arXiv preprint arXiv:1508.07909, 2015.
[26] Noam Shazeer, Azalia Mirhoseini, Krzysztof Maziarz, Andy Davis, Quoc Le, Geoffrey Hinton,



question, form a comprehensive prompt that empowers LLMs
to generate a well-informed answer.
The RAG research paradigm is continuously evolving, and
we categorize it into three stages: Naive RAG, Advanced
RAG, and Modular RAG, as showed in Figure 3. Despite
RAG method are cost-effective and surpass the performance
of the native LLM, they also exhibit several limitations.
The development of Advanced RAG and Modular RAG is
a response to these specific shortcomings in Naive RAG.
A. Naive RAG



Attention Is All Y ou Need
Ashish V aswani∗
Google Brain
avaswani@goog

In [ ]:
from langchain.document_transformers import EmbeddingsClusteringFilter, EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.document_transformers import LongContextReorder
from langchain.retrievers import ContextualCompressionRetriever
from re import search

In [ ]:
filter = EmbeddingsRedundantFilter(embeddings=hf_embeddings)
reordering = LongContextReorder()
pipeline = DocumentCompressorPipeline(
    transformers=[filter, reordering]
)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline,
    base_retriever=retriever,
    search_kwargs = {"k": 3, "include_metadata":True}
)

In [ ]:
docs = compression_retriever.get_relevant_documents("What is RAG?")
for doc in docs:
    print(doc.page_content)
    print("\n\n")

question, form a comprehensive prompt that empowers LLMs
to generate a well-informed answer.
The RAG research paradigm is continuously evolving, and
we categorize it into three stages: Naive RAG, Advanced
RAG, and Modular RAG, as showed in Figure 3. Despite
RAG method are cost-effective and surpass the performance
of the native LLM, they also exhibit several limitations.
The development of Advanced RAG and Modular RAG is
a response to these specific shortcomings in Naive RAG.
A. Naive RAG



opportunities for the development of RAG, enabling it to
address more complex problems and integrative or summary
questions that require reading a large amount of material to
answer [49]. Developing new RAG methods in the context of
super-long contexts is one of the future research trends.
B. RAG Robustness
The presence of noise or contradictory information during
retrieval can detrimentally affect RAG’s output quality. This
situation is figuratively referred to as “Misinformation can



marized in

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
from google.colab import userdata
groq_api = userdata.get('GROQ_API_KEY')

In [ ]:
llms = ChatGroq(
    model = "gemma2-9b-it",
    max_tokens= 1500,
    temperature= 0.75,
    streaming= True,
    verbose= True,
    api_key= groq_api,
)

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
chain = RetrievalQA.from_chain_type(
    llm=llms,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
)

In [ ]:
query = "What is a Rag?"
results = chain(query)

print(results["result"])
for doc in results["source_documents"]:
    print(doc.page_content)
    print("\n")

This text does not contain the answer to "What is a RAG?". 

However, it does state that:

*  RAG stands for "Retrieval-Augmented Generation"
*  It is a research paradigm that is continuously evolving.
*  It is categorized into three stages: Naive RAG, Advanced RAG, and Modular RAG. 


You may be able to find a more complete definition of RAG by looking elsewhere.  

question, form a comprehensive prompt that empowers LLMs
to generate a well-informed answer.
The RAG research paradigm is continuously evolving, and
we categorize it into three stages: Naive RAG, Advanced
RAG, and Modular RAG, as showed in Figure 3. Despite
RAG method are cost-effective and surpass the performance
of the native LLM, they also exhibit several limitations.
The development of Advanced RAG and Modular RAG is
a response to these specific shortcomings in Naive RAG.
A. Naive RAG


opportunities for the development of RAG, enabling it to
address more complex problems and integrative or summary
questions that requi